In [27]:
import re
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
from pylab import *
from decimal import Decimal
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
%matplotlib inline

In [3]:
def load_data(filename):
    ''' load data and plot the data and get the complexity of the data
    dictionary maps y to its feature vector x'''
    f = open(filename,'r')
    rows = 0
    colums = 0
    xy = defaultdict(list)
    for word in f:
        if word == " ":
            break
        if not word.startswith("#"):
            nums = word.split()
            nums = [float(n) for n in nums]
            xy[float(nums[-1])] = nums[0:len(nums)-1]
    return xy
                
                

In [8]:
sset1 = load_data("svar-set1.dat.txt")

In [ ]:
# sset2 = load_data("svar-set2.dat.txt")

In [ ]:
# sset3 = load_data("svar-set3.dat.txt")

In [ ]:
# sset4 = load_data("svar-set4.dat.txt")

In [ ]:
print len(sset1) , " " , len(sset2) , " " , len(sset3) , " " , len(sset4)

In [4]:
def plot_data(data,y_predict):
    '''This function takes a dictionary of the data loaded from the files'''
    x = list()
    y = list()
    # making a list of all feature values
    for d1 in data.values():
        for d2 in d1:
            x.append(d2)
    # making a list of all label values
    for d1 in data.keys():
        y.append(d1)
    plt.show()
    if (y_predict is not None):
        plt.plot(x,y,'ro')
        plt.plot(x,y_predict)
    else:
        plt.plot(x,y,'ro')
    plt.show()

In [ ]:
plot_data(sset1,None)

In [ ]:
plot_data(sset2,None)

In [ ]:
plot_data(sset3,None)

In [ ]:
plot_data(sset4,None)

In [5]:
def fit_single_feature_linear_model(data):
    '''fit linear model with the data
       compute training and testing error
       plot regression model on testing data'''
    x = list()
    y = data.keys()
    m = len(y)
    for d1 in data.values():
        for d2 in d1:
            x.append(d2)
    sum_of_xi = sum(x) 
    sum_of_xi2 = sum([d**2 for d in x])
    sum_of_yi = sum(y)
    sum_of_xiyi =  sum([x[i]*y[i] for i in range(len(x))])
    print 'm:', m
    print 'sum(x) : ' , sum_of_xi 
    print 'sum(x2): ' , sum_of_xi2
    print 'sum(y) : ', sum_of_yi 
    print 'sum(y2): ', sum_of_xiyi
    A = np.matrix([[m, sum_of_xi], [sum_of_xi, sum_of_xi2]])
    B = np.matrix([[sum_of_yi],[sum_of_xiyi]])
    theta = solve(A,B)
    return theta

In [ ]:
# theta_sset1 = fit_single_feature_linear_model(sset1)

In [ ]:
# theta_sset2 = fit_single_feature_linear_model(sset2)

In [ ]:
# theta_sset3 = fit_single_feature_linear_model(sset3)

In [ ]:
# theta_sset4 = fit_single_feature_linear_model(sset4)

In [13]:
def predict_y(x,theta):
#     theta0 = theta[0][0]
#     theta1 = theta[1][0]
#     print 'theta0', (theta0)
#     print 'theta1', (theta1)
#     x = list()
#     for d1 in data.values():
#         for d2 in d1:
#             x.append(d2)
    X = np.array(x)
    Y_predict = theta[0] + X*theta[1]
    return Y_predict

In [ ]:
# Y_sset1 = predict_y(sset1,
#                     np.array(theta_sset1))

In [ ]:
# Y_sset1

In [ ]:
# Y_sset2 = predict_y(sset2,np.array(theta_sset2))

In [ ]:
# Y_sset3 = predict_y(sset3,np.array(theta_sset3))

In [ ]:
# Y_sset4 = predict_y(sset4,np.array(theta_sset4))

In [ ]:
# plot_data(sset1,Y_sset1)
    

In [ ]:
# plot_data(sset2,Y_sset2)

In [ ]:
# plot_data(sset3,Y_sset3)

In [ ]:
# plot_data(sset4,Y_sset4)

In [25]:
def do_experiment(data,folds):
#     data = load_data(filename)
    # fit model and obtain theta value
    x = data.values()
    y = data.keys()
    theta = fit_single_feature_linear_model(data)
    Y_predict = predict_y(x,np.array(theta))
    train_err = 0
    test_err = 0
    
    for train_ind, test_ind in KFold(len(y), folds,shuffle=True):
        X_train = np.array(x)[train_ind]
        Y_predict_train = predict_y(X_train,np.array(theta))
        Y_train = np.array(y)[train_ind]
        train_error =  mean_squared_error(Y_train, Y_predict_train) 
        train_err += train_error
        
        X_test = np.array(x)[test_ind]
        Y_predict_test = predict_y(X_test,np.array(theta))
        Y_test = np.array(y)[test_ind]
        test_error = mean_squared_error(Y_test,Y_predict_test)
        test_err += test_error  
        
        print 'fold: ', folds, 'train_error: ', train_error ,'test_error: ', test_error
        print ' '
    train_err /= folds
    test_err /= folds
    print 'train_err: ', train_err, 'test_err' , test_err

In [26]:
do_experiment(sset1,10)

m: 200
sum(x) :  4000.0
sum(x2):  86733.6683417
sum(y) :  7996.65094195
sum(y2):  173306.774845
fold:  10 train_error:  4.30454614226 test_error:  3.5847908771
 
fold:  10 train_error:  4.06240646736 test_error:  5.76404795116
 
fold:  10 train_error:  4.38397551005 test_error:  2.86992656699
 
fold:  10 train_error:  4.40491731747 test_error:  2.68145030019
 
fold:  10 train_error:  4.25938128275 test_error:  3.99127461265
 
fold:  10 train_error:  4.27762713442 test_error:  3.8270619477
 
fold:  10 train_error:  4.15993059603 test_error:  4.88633079314
 
fold:  10 train_error:  4.3222607993 test_error:  3.42535896371
 
fold:  10 train_error:  3.98485910746 test_error:  6.46197419034
 
fold:  10 train_error:  4.16580180033 test_error:  4.83348995445
 
train_err:  4.23257061574 test_err 4.23257061574


In [ ]:
def compare_model():
    '''compare the regression model with the one created by python model'''
    regr = linear_model.LinearRegression()
    regr.fit(diabetes_X_train, diabetes_y_train)